**Шаг 1**. Загрузим и отобразим часть данных из каждого файла.

In [91]:
import pandas as pd
import numpy as np

groups = pd.read_csv('./data/groups.csv', sep=';')
print(groups.head(), end='\n'*2)

active_studs = pd.read_csv('./data/active_studs.csv', sep=';')
print(active_studs.head(), end='\n'*2)

checks = pd.read_csv('./data/checks.csv', sep=';')
print(checks.head(), end='\n'*2)
print(len(checks))

        id grp
0  1206794   B
1   331024   B
2  1733498   A
3   161574   B
4   168443   B

   student_id
0      581585
1     5723133
2     3276743
3     4238589
4     4475369

   student_id    rev
0        1627  990.0
1        3185  690.0
2       25973  690.0
3       26280  690.0
4      100300  990.0

541


**Шаг 2**. Объединим данные об активных пользователях и о размеченных группах в одну таблицу для того, чтобы понять, кто из активных студентов к какой группе относится.

In [92]:
active_studs_marked = pd.merge(
    active_studs,
    groups,
    how='left',
    left_on='student_id',
    right_on='id'
)
print(active_studs_marked.head())

   student_id       id grp
0      581585   581585   A
1     5723133  5723133   A
2     3276743  3276743   B
3     4238589  4238589   A
4     4475369  4475369   B


**Шаг 3**. Проверим, есть ли среди активных студентов те, кто не попал ни в одну группу.

In [93]:
print('No unmarked data' if len(active_studs_marked[active_studs_marked.id.notnull()]) == 
      len(active_studs_marked['id']) else 'Has unmarked data')

No unmarked data


**Шаг 4.** Объединим оставшуюся таблицу

In [94]:
all_data = pd.merge(
    active_studs_marked[['student_id', 'grp']],
    checks,
    how='left',
    left_on='student_id',
    right_on='student_id'
).reset_index(drop=True)
print(all_data.head())
print(all_data.count())

   student_id grp  rev
0      581585   A  NaN
1     5723133   A  NaN
2     3276743   B  NaN
3     4238589   A  NaN
4     4475369   B  NaN
student_id    8341
grp           8341
rev            392
dtype: int64


**Шаг 5**.
Начнем анализ. Для двух групп пользователей рассчитаем следующие показатели:
* CR из активности в оплату - при статистически значимом отличии конверсий можем утверждать, что изменение в интерфейсе стало причиной изменения поведения пользователей. 
* ARPAU/ARPPU - при увеличении среднего чека можем утверждать, что изменение положительно повлияло на юнит-экономику продукта.

**Шаг 5.1.** Рассчитаем CR из активного пользователя в покупку.

In [96]:
count_analysis = pd.merge(
    all_data.groupby('grp').student_id.count().reset_index().rename(columns={'student_id':'active'}),
    all_data.groupby('grp').rev.count().reset_index().rename(columns={'rev':'payed'}))
cr_analysis = count_analysis.copy()
cr_analysis['CR'] = cr_analysis['payed'] / cr_analysis['active']
print(cr_analysis)

  grp  active  payed        CR
0   A    1538     78  0.050715
1   B    6803    314  0.046156


**Анализ**: Проанализируем конверсии в покупку обоих групп при помощи доверительных интервалов.  
Можно воспользоваться формулой расчета доверительного интервала, но для упрощению использую калькулятор стат.значимости: https://mindbox.ru/ab-test-calculator/.  
**Результат**: Интерфейсные изменения не оказали значимого влияния на изменение конверсии.

**Шаг 5.2.** Рассчитаем ARPAU/ARPPU

In [97]:
check_analysis = all_data.groupby('grp').rev.sum().reset_index().rename(columns={'rev':'total_sum'})
check_analysis = pd.merge(
    count_analysis,
    check_analysis
)
check_analysis['ARPAU'] = check_analysis['total_sum'] / check_analysis['active']
check_analysis['ARPPU'] = check_analysis['total_sum'] / check_analysis['payed']
print(check_analysis)

  grp  active  payed    total_sum      ARPAU        ARPPU
0   A    1538     78   72820.0000  47.347204   933.589744
1   B    6803    314  394974.0035  58.058798  1257.878992


**Результат**. Интерфейсные изменения оказали влияние на увеличение среднего чека на обычного и на платящего пользователей.  
Таким образом, изменение позитивно повлияло на юнит-экономику в части прибыли.
При условии, что CAC останется тем же, изменение в целом позитивно повлияет на юнит-экономику продукта.

**Итоговые выводы**  
Интерфейсное изменение не оказало значимого влияния на конверсию в покупку, однако повысило прибыль с пользователя.  
При прочих равных условиях можем сказать, что выкатка изменения на всех пользователей позитивно повлияет на экономику продукта. 
  
  
**Для полноценного вывода отсутствуют данные**  
Представленные данные не позволяют полноценно судить о влиянии изменения на все продуктовые метрики.  
При реализации таких изменений стоит также оценивать влияние изменений на ключевые продуктовые показатели.  
Необходимо обратить внимание на следующие метрики: Retention в прохождение обучения после совершения оплаты, Retention в покупку, LTV пользователя, CAC и контрольный замер Retention по месяцам.